### J'importe mes bibliothèques

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import requests
import csv

### Je creer ma session PySpark en précisant que j'utilise un seul Worker (Puisque je travail en local)

In [2]:
def creerSession():
    ''' Cette fonction permet de creer une session spark si elle n'existe pas déjà '''
    spark = SparkSession.builder \
        .master("local[1]") \
        .appName("Traitement InfoClimat") \
        .getOrCreate()
    
    return spark

### Je lis mon fichier CSV dans Pyspark

Je précise qu'il n'y a pas de header et pas de schema de donnée

In [3]:
def lectureCSVSansHeader(nom_csv,spark):
    ''' Cette fonction lit un fichier CSV et le charge sur spark 
    @param quoi ? Type ?
    @return quoi ? Type ?
    @Except quoi ?
    '''
    mes_donnees = spark.read.format("csv")\
        .option("header", "false").option("inferSchema", "false")\
        .load("Data_brut/"+nom_csv)
    
    return mes_donnees

### Je récupère les ligne avec Les villes et leur code associé.

Je veux connaitre le nombre de villes dans mon fichier, pour cela je compte les ligne avec un # dedans.

Je veux connaitre le nombre de ville dans mon fichier. Pour cela je comptre le nombre de lignes et je retire 3.

* Une ligne avec METADATA
* Une ligne avec la source
* Une ligne avec la Licence

In [4]:
def recupere_lignes_inutiles(table_spark):
    ''' Cette fonction permet de recupere les lignes inutiles et de récuperer les lignes avec le nom des villes '''
    
    #Je requete toutes les lignes avec un # devant, qui sont les lignes avec un # devant
    lignes_diese = table_spark.select("_c0")\
                    .filter(col("_c0").contains("#"))
    
    #J'extrait les lignes dans une variable
    liste_lignes_diese = lignes_diese.collect()
    
    #Je compte le nombre de ville dans mon fichier
    nombre_de_villes = lignes_diese.count() - 3
    
    return nombre_de_villes, liste_lignes_diese



In [5]:
def recuperer_codes_ville(liste_lignes_dieses, nombre_de_villes):
    ''' Cette fontcion permet d'extraire les noms des villes et leur code et les stocker dans une liste associée, elle demande en entrée les lignes dièse et le nombre de villes '''
    
    # J'initialise ma liste vide
    liste_ville = []
    liste_code_et_ville=[]
    
    #Je fais une liste à partir de la ligne indice 1 et pour les lignes avec un nom de ville
    for ligne in liste_lignes_dieses[1:nombre_de_villes+1]:
        #Je cast mes lignes en str
        ligne = str(ligne)
        # Je retire la partie avant le #
        ligne_split = ligne.split("# ")
        # Je retire les deux derniers caractères de ma chaine de caractère ( correspond à : ') )
        # Sur l'element indice 1 de mon split, je récupère du début jusqu'au deux derniers caractères
        code_et_ville = ligne_split[1][:-2]
        #Je sépare le code de nom de la ville. Je creer une liste avec le code et le nom de la ville
        code_et_ville = code_et_ville.split(":")
        #J'extrait le nom de ma ville dans une variable, je supprime l'espasce à l'avant du nom de la ville.
        nom_de_ville = str(code_et_ville[1]).strip()
        #J'atoute ma liste code_et_ville à ma liste de codes et villes
        liste_code_et_ville.append(code_et_ville)
    
    return liste_code_et_ville

Avec une boucle for, je viens stocker les lignes avec le nom des ville et leur code dans une liste.

In [6]:
def listeVillesToDF(listeVilles,spark):
    columns = ["code","ville"]
    mon_dataframe = spark.createDataFrame(data = listeVilles, schema = columns)
    return mon_dataframe
    
    
    

Je stocke mes codes et mes villes dans un dictionnaire.

In [7]:
def villesdictionnaire(liste_villes_code, nombre_de_villes):
    ''' Cette fonction transforme ma liste de ville en dictionnaire '''
    villes = {liste_villes_code[i][0]:liste_villes_code[i][1] for i in range(nombre_de_villes)}
    return villes

### Je supprime les lignes qui ne m'interesse pas

J'applique un filtre sur mes données pour retirer les lignes qui ne m'interesse pas.

In [8]:
def tableSansInfo(table_spark):
    """ Cette fonction selectionne la table spark sans les lignes contenant des informations """

    table_sans_info = table_spark.select("_c0")\
            .filter(~col("_c0").contains("#") & ~col("_c0").contains("degC"))
    
    return table_sans_info

In [9]:
def exportCSV_uneColonne(table_spark):
    """ Cette fonction export une table en csv """
    table_spark.write.csv("./fichierTemporaire.csv", header = False, mode="overwrite")


In [10]:
def lectureFichierTemporaire(nom_csv):
    """ Cette fonction permet de lire un csv avec des header """
    donnees = spark.read.format("csv")\
        .option("header", "true").option("inferSchema", "true").option("delimiter", ";")\
        .load(nom_csv+".csv")
    return donnees


In [11]:
def lectureCSVavecHeader(spark):
    ''' Cette fonction lit un fichier CSV et le charge sur spark 
    @param quoi ? Type ?
    @return quoi ? Type ?
    @Except quoi ?
    '''
    mes_donnees = spark.read.format("csv")\
        .option("header", "true").option("inferSchema", "true")\
        .load("fichierTemporaire.csv")
    
    return mes_donnees

In [12]:
def remplacerCodeVille(table_spark,liste_code):
    """ Cette fonction remplace les codes présents en première colonne, par le nom des villes """
    nouvelle_table = table_spark.withColumn("Ville",\
                                            when(col("station_id").isin(liste_code.keys()),\
                                            remplacements[col("nom_colonne")]) \
                                            .otherwise(col("nom_colonne")))

In [13]:
def exportCSV_propre(table_spark):
    """ Cette fonction export une table en csv """
    table_spark.write.csv("./infoclimat.csv", header = True, mode="append")

In [14]:
def exportCodesVille(table_spark):
    """ Cette fonction export une table en csv """
    table_spark.write.csv("./codesVille.csv", header = True, mode="append")

In [15]:
# def testConnexion(reponse):
    
#     if reponse.status_code == 200:
#         print("Code :{}, la connexion api est établie".format(reponse.status_code))
#     else:
#         print("Code:{}, la connxeion api n'est pas établie".format(reponse.status_code))

In [16]:
# def requeteAPI(code,date_debut,date_fin,token):
#     '''
#     Cette fonction fait une requete API sur la plateforme infoclimat
#     Elle execute egalement la fonction testConnexion pour vérifier si la requete est réussie.
    
#     @param : code: str, date_debut : str, date_fin : str
#     @retrun : contenu de la page
#     '''
#     URL="https://www.infoclimat.fr/opendata/?method=get&format=csv&stations[]="+code+"&start="+date_debut+"&end="+date_fin+"&token="+token
    
#     reponse = requests.get(URL)
#     testConnexion(reponse)
    
#     return reponse

In [33]:
# def ecritureCSV(nomFichier,reponse):
#     with open("./Data_brut/"+nomFichier,'w',newline='') as fichier:
#         writer = csv.writer(fichier,delimiter=" ")
#         writer.writerow(reponse.text.splitlines(True))

In [51]:
def main():

    spark = creerSession()

    donnees_brute = lectureCSVSansHeader("ceciEstUnTest.csv",spark)

    nombreVille, listeLigneInutiles = recupere_lignes_inutiles(donnees_brute)

    codesVille = recuperer_codes_ville(listeLigneInutiles, nombreVille)
    
    codesVille_df = listeVillesToDF(codesVille,spark)
    
    exportCodesVille(codesVille_df)

    lignesUtiles = tableSansInfo(donnees_brute)

    exportCSV_uneColonne(lignesUtiles)

    tablePropre = lectureCSVavecHeader(spark)

    exportCSV_propre(tablePropre)

In [47]:
code = "000QW"
date_debut ="2020-01-01"
date_fin = "2020-12-31"
token = "Emp8A4J9Pk587RT3M5SwlhQ4jk3VtHVt0Qlq4XritihpvshdM7BVg"
nomFichier = code+"_"+date_debut+"_"+date_fin+".csv"

In [52]:
main()

In [ ]:
# Pour supprimer un dossier

# import shutil

# shutil.rmtree('infoclimat.csv')